# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig, Model

## Dataset

### Overview

The dataset used for this project can be obtained from the [Wine Quality- UCI Repo](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/). The dataset contains two seprate csv files for each the white and red wine qualities. Both dataset were combined and the red wine was assigned a value of 1 and white wine as 0 in the target column "y". The final merged csv file was upploaded to github and can be accessed [here](https://github.com/ammarahmed93/Capstone--Azure-Machine-Learning-Engineer/blob/main/wine-classification.csv). The data contains 6,598 rows and 13 columns including the target column. The goal is to predict the wine type (red or white wine) based on the other given properties such as pH, alchohol ...etc. Therefore this is a binary classification task.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'classification-automl'
project_folder = './capstoneProject'
experiment=Experiment(ws, experiment_name)

In [ ]:
found = False
description_text = "Wine Quality DataSet for Udacity Capstone Project"
key = 'wine-classification'

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    whiteWine_data = 'https://github.com/ammarahmed93/Capstone--Azure-Machine-Learning-Engineer/wine-classification.csv'
    dataset = Dataset.Tabular.from_delimited_files(whiteWine_data, separator=',')        
      
    #Register Dataset in Workspace
    dataset = whiteWine_dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

# Create or Attach an AmlCompute cluster

In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D16_v3',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 0)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

ref: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-cross-validation-data-splits

The following cell was used to assign the automl settings and configuration. For the automl settings the following was chosen:
- _n_cross_validations_ : The cross validation was provided to perform k-fold cross-validation. This parameter sets how many cross validations to perfotm, based on the same number of folds. In this case, 3 folds are used for cross-validation. Therefore 3 different trainings are performed with each training using 2/3 of the data and each validation using 1/3 of the data with a different holdout fold each time. The metrics are calculated with the average of the five validation metrics.
<br>
- _experiment_timeout_minutes_: In this case the maximum time that each iteration can run for is 20 minutes. if the iteration reaches the time limit then it will terminate.
<br>
- _enable_early_stopping_: When enabling early termination, if the score is not improving in the short term after the first 20 iterations then the experiment will terminate. 
<br>
- _max_concurrent_iterations_: The maximum number of iterations that would be execute in parallel, in this case 4 parallel iterations at max. This is dependent on the number of nodes provisioned and has to be one less than the provisioned nodes. 
<br>
- _primary_metric_: The metric that the AutoML will optimize for the model selection. The "AUC_weighted" was chosen in this case as its better metric for imblanced data.
<br>
- _task_: "classification" since its a binary classification task. 



In [12]:
# automl settings
automl_settings = {
    "n_cross_validations": 3,
    "experiment_timeout_minutes": 20,
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted'
}

#automl config 
automl_config = AutoMLConfig(compute_target=compute_target,
                             path = project_folder,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [13]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
classification-automl,AutoML_ef44b052-3c3b-41df-bbd1-7380ee53b8c6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
best_automl_run, best_automl_model = automl_run.get_output()
print(best_automl_run)
print(best_automl_model)

Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

Run(Experiment: classification-automl,
Id: AutoML_ef44b052-3c3b-41df-bbd1-7380ee53b8c6_36,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, tree_method='auto'))], verbose=False)), ('25', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced', criterion='entropy', max_depth=None, max_features=0.8, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.29368421052631577, min_samples_split=0.47842105263157897, min_we

In [16]:
best_automl_run.get_file_names()

# Download the yaml file that includes the environment dependencies
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [34]:
#TODO: Save the best model
best_automl_run.download_file('outputs/model.pkl', 'Automl_model.pkl')
# register best model
best_automl_run.register_model(model_name = "automl_VotingEnsemble.pkl", model_path = './outputs/')


Model(workspace=Workspace.create(name='quick-starts-ws-162664', subscription_id='b968fb36-f06a-4c76-a15f-afab68ae7667', resource_group='aml-quickstarts-162664'), name=automl_VotingEnsemble.pkl, id=automl_VotingEnsemble.pkl:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [37]:
# automl_model = best_automl_run.register_model(model_name = 'automl_VotingEnsemble', model_path='./outputs/')
# model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")
automl_model = automl_run.register_model(model_name = 'automl_VotingEnsemble.pkl')
print(automl_run.model_id)

automl_VotingEnsemble.pkl


In [39]:
print(automl_model.name, automl_model.id, automl_model.version, sep='\t')

automl_VotingEnsemble.pkl	automl_VotingEnsemble.pkl:2	2


In [40]:
environment = best_automl_run.get_environment()
# entry_script = 'source_dir/score.py'

inference_config = InferenceConfig(source_directory="./source_dir", entry_script="./score.py", environment=environment)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                     memory_gb = 1,
                                                     auth_enabled = True,
                                                     enable_app_insights = True)

service = Model.deploy(ws, 'wine-classification-aciservice', [automl_model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(show_output =True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-03 15:09:28+00:00 Creating Container Registry if not exists.
2021-11-03 15:09:28+00:00 Registering the environment.
2021-11-03 15:09:29+00:00 Use the existing image.
2021-11-03 15:09:30+00:00 Submitting deployment to compute.
2021-11-03 15:09:33+00:00 Checking the status of deployment wine-classification-aciservice..
2021-11-03 15:14:24+00:00 Checking the status of inference endpoint wine-classification-aciservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [43]:
df[df['y']==0]

,Column2,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,y
4898,0,7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5,0
4899,1,7.80,0.88,0.00,2.60,0.10,25.00,67.00,1.00,3.20,0.68,9.80,5,0
4900,2,7.80,0.76,0.04,2.30,0.09,15.00,54.00,1.00,3.26,0.65,9.80,5,0
4901,3,11.20,0.28,0.56,1.90,0.07,17.00,60.00,1.00,3.16,0.58,9.80,6,0
4902,4,7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,1594,6.20,0.60,0.08,2.00,0.09,32.00,44.00,0.99,3.45,0.58,10.50,5,0
6493,1595,5.90,0.55,0.10,2.20,0.06,39.00,51.00,1.00,3.52,0.76,11.20,6,0
6494,1596,6.30,0.51,0.13,2.30,0.08,29.00,40.00,1.00,3.42,0.75,11.00,6,0
6495,1597,5.90,0.65,0.12,2.00,0.07,32.00,44.00,1.00,3.57,0.71,10.20,5,0


TODO: In the cell below, send a request to the web service you deployed to test it.

In [49]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = { 
      "data": [ 
          {
              "Column2": "example_value",
              "fixed acidity": 0,
              "volatile acidity": 0,
              "citric acid": 0,
              "residual sugar": 0,
              "chlorides": 0,
              "free sulfur dioxide": 0,
              "total sulfur dioxide": 0,
              "density": 0,
              "pH": 0,
              "sulphates": 0,
              "alcohol": 0,
              "quality": 0
          }
      ]
 
}

body = str.encode(json.dumps(data))

url = 'http://13baaa78-384c-4473-bca9-aa3fc8f64722.southcentralus.azurecontainer.io/score'
api_key = 'Cabj58lVd2Ha8syeTBdQwg5sxvM8N3t9' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


b'"{\\"result\\": [1]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [48]:
# Check Model status
primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service primary authentication key: '+ primary)

Service state: Healthy
Service scoring URI: http://13baaa78-384c-4473-bca9-aa3fc8f64722.southcentralus.azurecontainer.io/score
Service primary authentication key: Cabj58lVd2Ha8syeTBdQwg5sxvM8N3t9


In [50]:
service.get_logs()

'2021-11-03T15:14:09,702717500+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2021-11-03T15:14:09,712726800+00:00 - iot-server/run \n2021-11-03T15:14:09,704358600+00:00 - rsyslog/run \n2021-11-03T15:14:09,768720500+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_705720c76ff57b57c77d577152dabb18/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-11-03T15:14:10,160170000+00:00 - iot-server/finish 1 0\n2021-11-03T15:14:10,161521300+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (70)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 98\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-11-03 15:14:13,624 | root | INFO | Starting up app insights client\

# Resource clean-up

In [51]:
service.delete()